# A Simple Scraper which accesses Job Postings and checks for specific strings

In [1]:
from bs4 import BeautifulSoup
import urllib.request as ur
import re
import sys

In [5]:
red_flags = ["senior", "intern", "contract", "staff"] #List of words to avoid in job title
#required = ["software"] #Can also check for required words

def qualifies(title):
    title = title.lower()
    #Define a function to check if a job title is worth checking out  
    for word in red_flags:
        if word in title: return False
    return True

#test:
qualifies("Senior Software Engineer")

False

In [15]:
# Now define the Regex, 
# 1. Should not have the phrase 1+ years, 1-2 Years, so on..
p1 = re.compile('[2-9]\s*\+?-?\s*[1-9]?\s*[yY]e?a?[rR][Ss]?')
# 2. Should not have mention of "Citizenship", "Citizens", so on..
p2 = re.compile('[Cc]itizens?(ship)?')

t1 = p1.search("2+ Years of experiencce")
t2 = p1.search("0-1 Year")
print (t1, "\n",t2)

<_sre.SRE_Match object; span=(0, 8), match='2+ Years'> 
 None


In [23]:
#The first page with search results
url_base = "base url here.. "
pgno = 0
try:
        response = ur.urlopen(url_base+str(pgno))
        html_doc = response.read()
except:
        print("URL not accesible")
        exit();
soup = BeautifulSoup(html_doc, 'html.parser')
"Ready."

'Ready.'

In [30]:
try:
    total_results = soup.find(id="searchCount").get_text()
    last_page = int(int(total_results[total_results.index("of")+2: total_results.index("jobs")].strip()) / 10) * 10
    print(last_page)
except:
    print ("No jobs found")

370


In [34]:
jobs_per_page = 10
goodlinks = []
for pgno in range(0,last_page,jobs_per_page):
    if pgno > 0:
        try:
            response = ur.urlopen(url_base+str(pgno))
            html_doc = response.read()
        except:
            break;
        soup = BeautifulSoup(html_doc, 'html.parser')
    for job in soup.find_all(class_='result'):
        link = job.find(class_="turnstileLink")
        try:
            jt = link.get('title')
        except:
            jt = ""
        try:
            comp = job.find(class_='company').get_text().strip()
        except:
            comp = ""

        if(qualifies(jt.lower())):
            toVisit = "http://www.indeed.com"+link.get('href')
            try:
                html_doc = ur.urlopen(toVisit).read().decode('utf-8')
            except:
                continue;
            m = p1.search(html_doc)
            n = p2.search(html_doc)
            if not m and not n:
                print(jt,",",comp,":",toVisit,"\n")
                goodlinks.append(toVisit)

Software Engineer, University New Graduate , Houzz : http://www.indeed.com/rc/clk?jk=b13f83b8083235b1&fccid=cd808c272e6d956a 

Software Engineer , LiveAction : http://www.indeed.com/rc/clk?jk=02ecac2491ae61cf&fccid=ace8b9c0b5f18b27 

Java Developer (Full Time) , Ezen computer services : http://www.indeed.com/company/Ezen-computer-services-INC/jobs/Java-Developer-c84007a1deb5b12d?fccid=8db39aaae3efa27e 

Software Engineer, Cloud Platform - UI , Toyota Research Institute : http://www.indeed.com/rc/clk?jk=d16fb14301cdff9f&fccid=e490ccf806951166 

Dev-Ops / Java Software Engineer , Paypal : http://www.indeed.com/rc/clk?jk=31fe013ea2b5fe01&fccid=978d9fd9799d55a8 

Jr. Web Application Developer (AMRD2017) , Fortinet : http://www.indeed.com/rc/clk?jk=eb7835743224b1db&fccid=28d99c55a9ebe7b6 



And that's all, very easy to customize for any kind of task.